In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('src')
# !pip install -qq .

In [ ]:
import os
import json
from PIL import Image
from glasses_detector import GlassesClassifier, GlassesDetector, GlassesSegmenter

In [ ]:
def display_images_in_row(directory, padding=10):
    # Load all images in the directory
    images = [Image.open(os.path.join(directory, img)) for img in os.listdir(directory)]

    # Create a new image with enough width to hold all images and padding
    total_width = sum(img.width for img in images) + padding * (len(images) - 1)
    max_height = max(img.height for img in images)
    new_img = Image.new('RGB', (total_width, max_height))

    # Paste images into new image with padding
    x = 0
    for img in images:
        new_img.paste(img, (x, 0))
        x += img.width + padding

    new_img.show()

## Classification

### Command Line

Let's first run the simplest prediction to see if the person is wearing glasses in `demo/0.jpg`. By default, if we don't specify the _task_ `kind`, it will default to `anyglasses` for _classification_.

In [ ]:
!python -m src.glasses_detector --input data/demo/0.jpg --task classification -w checkpoints/classification/classification-anyglasses-medium.pth

Let's now run predict if _sunglasses_ are present in every image under `demo/`. In this example, let's choose to save to a single `csv` file and encode predictions as integers.

In [ ]:
!python -m src.glasses_detector -i data/demo -o is_sunglasses.csv --format int --task classification:sunglasses -w checkpoints/classification/classification-sunglasses-medium.pth
!cat is_sunglasses.csv

### Python Script

Here we perform the same prediction considering _anyglasses_ on `demo/1.jpg`. This cell simply shows how to use `process_file` with some _format_ examples. Note that it is also possible to specify `output_path`, specify a list of paths for `input_path`, or, if you only need to return the predictions without showing/saving the results, `predict` method could be helpful.

In [ ]:
# Load the model for anyglasses classification
cls_anyglasses = GlassesClassifier(kind="anyglasses", weights='checkpoints/classification/classification-anyglasses-medium.pth')

def my_format(img, pred):
    # Define a custom format function (custom label and draw on image)
    label = "Wears glasses" if (pred > 0).item() else "Does not wear"
    return GlassesClassifier.draw_label(img, label)

# Process the image with various formats (disable the return value with _)
_ = cls_anyglasses.process_file("data/demo/1.jpg", show=True, format="logit")
_ = cls_anyglasses.process_file("data/demo/1.jpg", show=True, format={True: "yes", False: -1})
_ = cls_anyglasses.process_file("data/demo/1.jpg", show=True, format=my_format)

This cell shows how to use `process_dir` to save the predictions for each image into a single file and a directory of multiple files (one for each prediction). Note how `cls_sunglasses` can also be just called directly to confirm the presence of _sunglasses_ in `[0.jpg, 3.jpg]`.

In [ ]:
# Load the model for sunglasses classification
cls_sunglasses = GlassesClassifier(kind="sunglasses", weights='checkpoints/classification/classification-sunglasses-medium.pth')

# Process a directory of images and save the results to a json file as well as to a directory as images
cls_sunglasses.process_dir("data/demo", "is_sunglasses.json", format="proba", batch_size=3, pbar=False)
cls_sunglasses.process_dir("data/demo", "is_sunglasses", format="img", batch_size=6, pbar=False)

# Confirm "0.jpg" and "3.jpg" are indeed sunglasses, print predictions on the whole dir
print("[0.jpg, 3.jpg]:", cls_sunglasses(image=["data/demo/0.jpg", "data/demo/3.jpg"]))
print(json.load(open("is_sunglasses.json")))
display_images_in_row("is_sunglasses")

## Segmentation